# Initial Analysis

#### Imports

In [2]:
import sys
import math
import warnings

import psycopg2
import wrds
import gzip

import seaborn as sns
import os
import quandl
import json
import zipfile
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import functools
import requests
import io

import urllib.request
from urllib.error import HTTPError
# from html_table_parser.parser import HTMLTableParser
from bs4 import BeautifulSoup
import re

import plotnine as p9
from plotnine import ggplot, scale_x_date, guides, guide_legend, geom_bar, scale_y_continuous, \
    scale_color_identity, geom_line, geom_point, labs, theme_minimal, theme, element_blank, element_text, \
        geom_ribbon, geom_hline, aes, scale_size_manual, scale_color_manual, ggtitle

from datetime import datetime
import datetime

import pandas as pd
# import pandas_market_calendars as mcal
from pandas.plotting import autocorrelation_plot
import numpy as np
from numpy import cumsum, log, polyfit, sqrt, std, subtract
import scipy as sp
from scipy.stats import norm
import scipy.stats as stats

from statsmodels.tsa.stattools import coint
from statsmodels.graphics.tsaplots import plot_acf
import statsmodels.api as sm
from statsmodels.tsa.stattools import acf
from statsmodels.regression.linear_model import OLS
from statsmodels.tools.tools import add_constant

from collections import deque
from bisect import insort, bisect_left
from itertools import islice


### Retrieve as Variable

In [ ]:
csv_file_path = 'option_data.csv'
spydata = pd.read_csv(csv_file_path)

In [ ]:
display(spydata)

,date,exdate,last_date,cp_flag,strike_price,best_bid,best_offer,volume,open_interest,impl_volatility,delta,gamma,vega,theta,expiry_indicator
0,2018-01-02,2018-01-03,2017-12-28,C,235000.0,33.59,33.81,0.0,187.0,NaN,NaN,NaN,NaN,NaN,w
1,2018-01-02,2018-01-03,2018-01-02,C,240000.0,28.59,28.76,1.0,88.0,NaN,NaN,NaN,NaN,NaN,w
2,2018-01-02,2018-01-03,2017-12-27,C,242500.0,26.09,26.32,0.0,2.0,NaN,NaN,NaN,NaN,NaN,w
3,2018-01-02,2018-01-03,2018-01-02,C,245000.0,23.59,23.81,12.0,58.0,NaN,NaN,NaN,NaN,NaN,w
4,2018-01-02,2018-01-03,NaN,C,247500.0,21.08,21.32,0.0,0.0,NaN,NaN,NaN,NaN,NaN,w
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10448189,2023-02-28,2025-03-21,2022-12-28,P,600000.0,200.50,205.50,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
10448190,2023-02-28,2025-03-21,NaN,P,605000.0,205.50,210.50,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
10448191,2023-02-28,2025-03-21,NaN,P,610000.0,210.00,215.00,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
10448192,2023-02-28,2025-03-21,NaN,P,615000.0,215.00,220.00,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


Order by the appropriate date, then expiry date, then strike price within the date spread.

In [ ]:
spydata_sorted = spydata.sort_values(by=['date', 'exdate', 'cp_flag', 'strike_price'])
#spydata_cleaned = spydata_sorted.dropna(subset=['impl_volatility', 'delta', 'gamma', 'vega', 'theta'])
#spydata_final = spydata_cleaned.reset_index(drop=True)
spydata_final = spydata_sorted.reset_index(drop=True)

In [ ]:
display(spydata_final)

,date,exdate,last_date,cp_flag,strike_price,best_bid,best_offer,volume,open_interest,impl_volatility,delta,gamma,vega,theta,expiry_indicator
0,2018-01-02,2018-01-03,2017-12-28,C,235000.0,33.59,33.81,0.0,187.0,NaN,NaN,NaN,NaN,NaN,w
1,2018-01-02,2018-01-03,2018-01-02,C,240000.0,28.59,28.76,1.0,88.0,NaN,NaN,NaN,NaN,NaN,w
2,2018-01-02,2018-01-03,2017-12-27,C,242500.0,26.09,26.32,0.0,2.0,NaN,NaN,NaN,NaN,NaN,w
3,2018-01-02,2018-01-03,2018-01-02,C,245000.0,23.59,23.81,12.0,58.0,NaN,NaN,NaN,NaN,NaN,w
4,2018-01-02,2018-01-03,NaN,C,247500.0,21.08,21.32,0.0,0.0,NaN,NaN,NaN,NaN,NaN,w
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10448189,2023-02-28,2025-03-21,2022-12-28,P,600000.0,200.50,205.50,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
10448190,2023-02-28,2025-03-21,NaN,P,605000.0,205.50,210.50,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
10448191,2023-02-28,2025-03-21,NaN,P,610000.0,210.00,215.00,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
10448192,2023-02-28,2025-03-21,NaN,P,615000.0,215.00,220.00,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
spydata_final.to_csv('sorteddata.csv', index=False)

Rather than download again, simply reload.

In [ ]:
spy_optiondata = pd.read_csv('sorteddata.csv')

In [ ]:
display(spy_optiondata)

,date,exdate,last_date,cp_flag,strike_price,best_bid,best_offer,volume,open_interest,impl_volatility,delta,gamma,vega,theta,expiry_indicator
0,2018-01-02,2018-01-03,2017-12-28,C,235000.0,33.59,33.81,0.0,187.0,NaN,NaN,NaN,NaN,NaN,w
1,2018-01-02,2018-01-03,2018-01-02,C,240000.0,28.59,28.76,1.0,88.0,NaN,NaN,NaN,NaN,NaN,w
2,2018-01-02,2018-01-03,2017-12-27,C,242500.0,26.09,26.32,0.0,2.0,NaN,NaN,NaN,NaN,NaN,w
3,2018-01-02,2018-01-03,2018-01-02,C,245000.0,23.59,23.81,12.0,58.0,NaN,NaN,NaN,NaN,NaN,w
4,2018-01-02,2018-01-03,NaN,C,247500.0,21.08,21.32,0.0,0.0,NaN,NaN,NaN,NaN,NaN,w
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10448189,2023-02-28,2025-03-21,2022-12-28,P,600000.0,200.50,205.50,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
10448190,2023-02-28,2025-03-21,NaN,P,605000.0,205.50,210.50,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
10448191,2023-02-28,2025-03-21,NaN,P,610000.0,210.00,215.00,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
10448192,2023-02-28,2025-03-21,NaN,P,615000.0,215.00,220.00,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


Now we'll load the ticker price data, directly from local.

In [ ]:
spy_tickerdata = pd.read_csv('spy_tickerdata.csv')

In [ ]:
display(spy_tickerdata)

,ticker,date,open,high,low,close,volume,dividend,split,adj_open,adj_high,adj_low,adj_close,adj_volume
0,SPY,1993-08-13,45.093700,45.156200,45.093700,45.125000,103500.0,0.0,1.0,25.778428,25.814157,25.778428,25.796321,103500.0
1,SPY,1993-08-16,45.156200,45.500000,45.156200,45.375000,241800.0,0.0,1.0,25.814157,26.010695,25.814157,25.939237,241800.0
2,SPY,1993-08-17,45.343700,45.531200,45.343700,45.531200,369300.0,0.0,1.0,25.921344,26.028531,25.921344,26.028531,369300.0
3,SPY,1993-08-18,45.687500,45.875000,45.656200,45.781200,414300.0,0.0,1.0,26.117882,26.225069,26.099989,26.171447,414300.0
4,SPY,1993-08-19,45.812500,45.812500,45.718700,45.781200,28500.0,0.0,1.0,26.189340,26.189340,26.135718,26.171447,28500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7814,SPY,1993-06-22,44.656200,44.656200,44.562500,44.625000,137500.0,0.0,1.0,25.528325,25.528325,25.474761,25.510490,137500.0
7815,SPY,1994-11-16,46.765598,46.843700,46.609299,46.843700,106900.0,0.0,1.0,27.589587,27.635663,27.497377,27.635663,106900.0
7816,SPY,2010-03-10,114.510002,115.279999,114.410004,114.970001,186088800.0,0.0,1.0,87.964535,88.556033,87.887719,88.317898,186088800.0
7817,SPY,2022-02-15,443.730000,446.280000,443.180000,446.100000,88435136.0,0.0,1.0,430.043935,432.515285,429.510899,432.340837,88435136.0


## Adjustments

Make some adjustments and attach to our data table for options.

In [ ]:
spy_tickerdata['date'] = pd.to_datetime(spy_tickerdata['date'])
start_date = '2018-01-01'
end_date = '2023-02-28'
spy_tickerdata_filtered = spy_tickerdata[(spy_tickerdata['date'] >= start_date) & (spy_tickerdata['date'] <= end_date)]

spy_tickerdata_sorted = spy_tickerdata_filtered.sort_values(by='date')
spy_tickerdata_reduced = spy_tickerdata_sorted[['date', 'close', 'adj_open', 'adj_close', 'adj_volume']]

In [ ]:
spy_optiondata['date'] = pd.to_datetime(spy_optiondata['date'])
spy_optiondata['strike_price'] = spy_optiondata['strike_price'] / 1000.0

In [ ]:
spy_optiondata_enriched = pd.merge(spy_optiondata, spy_tickerdata_reduced, on='date', how='left')

In [ ]:
display(spy_optiondata_enriched)

,date,exdate,last_date,cp_flag,strike_price,best_bid,best_offer,volume,open_interest,impl_volatility,delta,gamma,vega,theta,expiry_indicator,close,adj_open,adj_close,adj_volume
0,2018-01-02,2018-01-03,2017-12-28,C,235.0,33.59,33.81,0.0,187.0,NaN,NaN,NaN,NaN,NaN,w,268.77,242.053393,242.893856,86655749.0
1,2018-01-02,2018-01-03,2018-01-02,C,240.0,28.59,28.76,1.0,88.0,NaN,NaN,NaN,NaN,NaN,w,268.77,242.053393,242.893856,86655749.0
2,2018-01-02,2018-01-03,2017-12-27,C,242.5,26.09,26.32,0.0,2.0,NaN,NaN,NaN,NaN,NaN,w,268.77,242.053393,242.893856,86655749.0
3,2018-01-02,2018-01-03,2018-01-02,C,245.0,23.59,23.81,12.0,58.0,NaN,NaN,NaN,NaN,NaN,w,268.77,242.053393,242.893856,86655749.0
4,2018-01-02,2018-01-03,NaN,C,247.5,21.08,21.32,0.0,0.0,NaN,NaN,NaN,NaN,NaN,w,268.77,242.053393,242.893856,86655749.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10448189,2023-02-28,2025-03-21,2022-12-28,P,600.0,200.50,205.50,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,396.26,391.240559,390.285185,96141367.0
10448190,2023-02-28,2025-03-21,NaN,P,605.0,205.50,210.50,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,396.26,391.240559,390.285185,96141367.0
10448191,2023-02-28,2025-03-21,NaN,P,610.0,210.00,215.00,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,396.26,391.240559,390.285185,96141367.0
10448192,2023-02-28,2025-03-21,NaN,P,615.0,215.00,220.00,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,396.26,391.240559,390.285185,96141367.0


In [ ]:
spy_optiondata_enriched.to_csv('combinedata.csv', index=False)

In [ ]:
spydata_cleaned = spy_optiondata_enriched.dropna(subset=['impl_volatility', 'delta', 'gamma', 'vega', 'theta'])
spydata_dropna = spydata_cleaned.reset_index(drop=True)
spydata_dropna.to_csv('combinedata_dropna.csv', index=False)

## Load useful data

In [3]:
filepath = 'combinedata.csv'
filepath1 = 'combinedata_dropna.csv'
options_df = pd.read_csv(filepath)
options_df1 = pd.read_csv(filepath1)

In [4]:
display(options_df)

,date,exdate,last_date,cp_flag,strike_price,best_bid,best_offer,volume,open_interest,impl_volatility,delta,gamma,vega,theta,expiry_indicator,close,adj_open,adj_close,adj_volume
0,2018-01-02,2018-01-03,2017-12-28,C,235.0,33.59,33.81,0.0,187.0,NaN,NaN,NaN,NaN,NaN,w,268.77,242.053393,242.893856,86655749.0
1,2018-01-02,2018-01-03,2018-01-02,C,240.0,28.59,28.76,1.0,88.0,NaN,NaN,NaN,NaN,NaN,w,268.77,242.053393,242.893856,86655749.0
2,2018-01-02,2018-01-03,2017-12-27,C,242.5,26.09,26.32,0.0,2.0,NaN,NaN,NaN,NaN,NaN,w,268.77,242.053393,242.893856,86655749.0
3,2018-01-02,2018-01-03,2018-01-02,C,245.0,23.59,23.81,12.0,58.0,NaN,NaN,NaN,NaN,NaN,w,268.77,242.053393,242.893856,86655749.0
4,2018-01-02,2018-01-03,NaN,C,247.5,21.08,21.32,0.0,0.0,NaN,NaN,NaN,NaN,NaN,w,268.77,242.053393,242.893856,86655749.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10448189,2023-02-28,2025-03-21,2022-12-28,P,600.0,200.50,205.50,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,396.26,391.240559,390.285185,96141367.0
10448190,2023-02-28,2025-03-21,NaN,P,605.0,205.50,210.50,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,396.26,391.240559,390.285185,96141367.0
10448191,2023-02-28,2025-03-21,NaN,P,610.0,210.00,215.00,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,396.26,391.240559,390.285185,96141367.0
10448192,2023-02-28,2025-03-21,NaN,P,615.0,215.00,220.00,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,396.26,391.240559,390.285185,96141367.0


In [5]:
display(options_df1)

,date,exdate,last_date,cp_flag,strike_price,best_bid,best_offer,volume,open_interest,impl_volatility,delta,gamma,vega,theta,expiry_indicator,close,adj_open,adj_close,adj_volume
0,2018-01-02,2018-01-03,2018-01-02,C,267.5,1.28,1.34,5514.0,7184.0,0.062960,0.926657,0.157172,1.959172,-26.221690,w,268.77,242.053393,242.893856,86655749.0
1,2018-01-02,2018-01-03,2018-01-02,C,268.0,0.85,0.88,14476.0,5520.0,0.060924,0.819616,0.306547,3.697382,-44.389840,w,268.77,242.053393,242.893856,86655749.0
2,2018-01-02,2018-01-03,2018-01-02,C,268.5,0.48,0.50,12516.0,5895.0,0.058946,0.633232,0.453884,5.294787,-59.509580,w,268.77,242.053393,242.893856,86655749.0
3,2018-01-02,2018-01-03,2018-01-02,C,269.0,0.21,0.22,14745.0,5761.0,0.055630,0.390490,0.490317,5.399054,-56.377560,w,268.77,242.053393,242.893856,86655749.0
4,2018-01-02,2018-01-03,2018-01-02,C,269.5,0.07,0.08,1742.0,7512.0,0.055298,0.178472,0.335492,3.670619,-37.772850,w,268.77,242.053393,242.893856,86655749.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9261854,2023-02-28,2025-03-21,2023-02-08,P,450.0,57.50,62.50,0.0,75.0,0.161451,-0.714413,0.007297,154.462000,1.957872,NaN,396.26,391.240559,390.285185,96141367.0
9261855,2023-02-28,2025-03-21,2023-01-09,P,455.0,61.00,66.00,0.0,0.0,0.158546,-0.758524,0.007626,151.188600,2.874898,NaN,396.26,391.240559,390.285185,96141367.0
9261856,2023-02-28,2025-03-21,2023-01-09,P,460.0,64.50,69.50,0.0,14.0,0.154637,-0.805469,0.007817,123.774400,4.337152,NaN,396.26,391.240559,390.285185,96141367.0
9261857,2023-02-28,2025-03-21,2023-01-09,P,465.0,68.50,73.50,0.0,21.0,0.152118,-0.846083,0.007509,79.376680,6.359011,NaN,396.26,391.240559,390.285185,96141367.0


## Quick Look

### Combined dataset:

In [6]:
exdates = set(options_df['exdate'])
quotedates = set(options_df['date'])

In [7]:
exdates - quotedates

{'2018-12-05',
 '2023-03-01',
 '2023-03-02',
 '2023-03-03',
 '2023-03-06',
 '2023-03-07',
 '2023-03-08',
 '2023-03-09',
 '2023-03-10',
 '2023-03-13',
 '2023-03-14',
 '2023-03-17',
 '2023-03-24',
 '2023-03-31',
 '2023-04-06',
 '2023-04-21',
 '2023-05-19',
 '2023-06-16',
 '2023-06-30',
 '2023-07-21',
 '2023-08-18',
 '2023-09-15',
 '2023-09-29',
 '2023-12-15',
 '2023-12-29',
 '2024-01-19',
 '2024-03-15',
 '2024-06-21',
 '2024-12-20',
 '2025-01-17',
 '2025-03-21'}

In [8]:
quotedates - exdates

{'2018-01-02',
 '2018-01-04',
 '2018-01-08',
 '2018-01-09',
 '2018-01-11',
 '2018-01-16',
 '2018-01-18',
 '2018-01-22',
 '2018-01-23',
 '2018-01-25',
 '2018-01-29',
 '2018-01-30',
 '2018-02-01',
 '2018-02-05',
 '2018-02-06',
 '2018-02-08',
 '2018-02-12',
 '2018-02-13',
 '2018-02-15',
 '2018-02-20',
 '2018-02-22',
 '2018-02-27',
 '2018-03-01',
 '2018-03-06',
 '2018-03-08',
 '2018-03-13',
 '2018-03-15',
 '2018-03-20',
 '2018-03-22',
 '2018-03-27',
 '2018-04-03',
 '2018-04-05',
 '2018-04-10',
 '2018-04-12',
 '2018-04-17',
 '2018-04-19',
 '2018-04-24',
 '2018-04-26',
 '2018-05-01',
 '2018-05-03',
 '2018-05-08',
 '2018-05-10',
 '2018-05-15',
 '2018-05-17',
 '2018-05-22',
 '2018-05-24',
 '2018-05-31',
 '2018-06-05',
 '2018-06-07',
 '2018-06-12',
 '2018-06-14',
 '2018-06-19',
 '2018-06-21',
 '2018-06-26',
 '2018-06-28',
 '2018-07-05',
 '2018-07-10',
 '2018-07-12',
 '2018-07-17',
 '2018-07-19',
 '2018-07-24',
 '2018-07-26',
 '2018-07-31',
 '2018-08-02',
 '2018-08-07',
 '2018-08-09',
 '2018-08-

In [9]:
exdates & quotedates

{'2018-01-03',
 '2018-01-05',
 '2018-01-10',
 '2018-01-12',
 '2018-01-17',
 '2018-01-19',
 '2018-01-24',
 '2018-01-26',
 '2018-01-31',
 '2018-02-02',
 '2018-02-07',
 '2018-02-09',
 '2018-02-14',
 '2018-02-16',
 '2018-02-21',
 '2018-02-23',
 '2018-02-26',
 '2018-02-28',
 '2018-03-02',
 '2018-03-05',
 '2018-03-07',
 '2018-03-09',
 '2018-03-12',
 '2018-03-14',
 '2018-03-16',
 '2018-03-19',
 '2018-03-21',
 '2018-03-23',
 '2018-03-26',
 '2018-03-28',
 '2018-03-29',
 '2018-04-02',
 '2018-04-04',
 '2018-04-06',
 '2018-04-09',
 '2018-04-11',
 '2018-04-13',
 '2018-04-16',
 '2018-04-18',
 '2018-04-20',
 '2018-04-23',
 '2018-04-25',
 '2018-04-27',
 '2018-04-30',
 '2018-05-02',
 '2018-05-04',
 '2018-05-07',
 '2018-05-09',
 '2018-05-11',
 '2018-05-14',
 '2018-05-16',
 '2018-05-18',
 '2018-05-21',
 '2018-05-23',
 '2018-05-25',
 '2018-05-29',
 '2018-05-30',
 '2018-06-01',
 '2018-06-04',
 '2018-06-06',
 '2018-06-08',
 '2018-06-11',
 '2018-06-13',
 '2018-06-15',
 '2018-06-18',
 '2018-06-20',
 '2018-06-

In [10]:
options_df['TTE'] = (pd.to_datetime(options_df['exdate']) - pd.to_datetime(options_df['date'])).dt.days
options_df

,date,exdate,last_date,cp_flag,strike_price,best_bid,best_offer,volume,open_interest,impl_volatility,delta,gamma,vega,theta,expiry_indicator,close,adj_open,adj_close,adj_volume,TTE
0,2018-01-02,2018-01-03,2017-12-28,C,235.0,33.59,33.81,0.0,187.0,NaN,NaN,NaN,NaN,NaN,w,268.77,242.053393,242.893856,86655749.0,1
1,2018-01-02,2018-01-03,2018-01-02,C,240.0,28.59,28.76,1.0,88.0,NaN,NaN,NaN,NaN,NaN,w,268.77,242.053393,242.893856,86655749.0,1
2,2018-01-02,2018-01-03,2017-12-27,C,242.5,26.09,26.32,0.0,2.0,NaN,NaN,NaN,NaN,NaN,w,268.77,242.053393,242.893856,86655749.0,1
3,2018-01-02,2018-01-03,2018-01-02,C,245.0,23.59,23.81,12.0,58.0,NaN,NaN,NaN,NaN,NaN,w,268.77,242.053393,242.893856,86655749.0,1
4,2018-01-02,2018-01-03,NaN,C,247.5,21.08,21.32,0.0,0.0,NaN,NaN,NaN,NaN,NaN,w,268.77,242.053393,242.893856,86655749.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10448189,2023-02-28,2025-03-21,2022-12-28,P,600.0,200.50,205.50,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,396.26,391.240559,390.285185,96141367.0,752
10448190,2023-02-28,2025-03-21,NaN,P,605.0,205.50,210.50,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,396.26,391.240559,390.285185,96141367.0,752
10448191,2023-02-28,2025-03-21,NaN,P,610.0,210.00,215.00,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,396.26,391.240559,390.285185,96141367.0,752
10448192,2023-02-28,2025-03-21,NaN,P,615.0,215.00,220.00,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,396.26,391.240559,390.285185,96141367.0,752


In [11]:
filtered_df = options_df[(options_df['TTE'] >= 27) & (options_df['TTE'] <= 33)]
filtered_df

,date,exdate,last_date,cp_flag,strike_price,best_bid,best_offer,volume,open_interest,impl_volatility,delta,gamma,vega,theta,expiry_indicator,close,adj_open,adj_close,adj_volume,TTE
972,2018-01-02,2018-01-31,NaN,C,165.0,103.82,104.01,0.0,0.0,NaN,NaN,NaN,NaN,NaN,w,268.77,242.053393,242.893856,86655749.0,29
973,2018-01-02,2018-01-31,2017-12-28,C,240.0,29.06,29.26,0.0,12.0,0.193414,0.983311,0.002835,3.151210,-7.538074,w,268.77,242.053393,242.893856,86655749.0,29
974,2018-01-02,2018-01-31,2017-12-28,C,245.0,24.12,24.32,0.0,2.0,0.175939,0.972425,0.004764,4.816286,-9.064763,w,268.77,242.053393,242.893856,86655749.0,29
975,2018-01-02,2018-01-31,2017-12-28,C,250.0,19.20,19.38,0.0,152.0,0.153960,0.957185,0.007811,6.907079,-10.439730,w,268.77,242.053393,242.893856,86655749.0,29
976,2018-01-02,2018-01-31,2017-12-28,C,255.0,14.33,14.52,0.0,52.0,0.133378,0.926605,0.013784,10.547560,-12.556030,w,268.77,242.053393,242.893856,86655749.0,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10444209,2023-02-28,2023-03-31,2022-06-23,P,570.0,172.26,173.45,0.0,0.0,NaN,NaN,NaN,NaN,NaN,m,396.26,391.240559,390.285185,96141367.0,31
10444210,2023-02-28,2023-03-31,2022-11-18,P,575.0,177.25,178.44,0.0,0.0,NaN,NaN,NaN,NaN,NaN,m,396.26,391.240559,390.285185,96141367.0,31
10444211,2023-02-28,2023-03-31,2022-06-23,P,580.0,182.23,183.43,0.0,0.0,NaN,NaN,NaN,NaN,NaN,m,396.26,391.240559,390.285185,96141367.0,31
10444212,2023-02-28,2023-03-31,2022-11-15,P,585.0,187.22,188.42,0.0,0.0,NaN,NaN,NaN,NaN,NaN,m,396.26,391.240559,390.285185,96141367.0,31


### Dropna dataset:

In [12]:
exdates1 = set(options_df1['exdate'])
quotedates1 = set(options_df1['date'])

In [13]:
len(exdates1 - quotedates1)

31

In [14]:
len(quotedates1 - exdates1)

468

In [15]:
len(exdates1 & quotedates1)

830

In [16]:
options_df1['TTE'] = (pd.to_datetime(options_df1['exdate']) - pd.to_datetime(options_df1['date'])).dt.days
options_df1

,date,exdate,last_date,cp_flag,strike_price,best_bid,best_offer,volume,open_interest,impl_volatility,delta,gamma,vega,theta,expiry_indicator,close,adj_open,adj_close,adj_volume,TTE
0,2018-01-02,2018-01-03,2018-01-02,C,267.5,1.28,1.34,5514.0,7184.0,0.062960,0.926657,0.157172,1.959172,-26.221690,w,268.77,242.053393,242.893856,86655749.0,1
1,2018-01-02,2018-01-03,2018-01-02,C,268.0,0.85,0.88,14476.0,5520.0,0.060924,0.819616,0.306547,3.697382,-44.389840,w,268.77,242.053393,242.893856,86655749.0,1
2,2018-01-02,2018-01-03,2018-01-02,C,268.5,0.48,0.50,12516.0,5895.0,0.058946,0.633232,0.453884,5.294787,-59.509580,w,268.77,242.053393,242.893856,86655749.0,1
3,2018-01-02,2018-01-03,2018-01-02,C,269.0,0.21,0.22,14745.0,5761.0,0.055630,0.390490,0.490317,5.399054,-56.377560,w,268.77,242.053393,242.893856,86655749.0,1
4,2018-01-02,2018-01-03,2018-01-02,C,269.5,0.07,0.08,1742.0,7512.0,0.055298,0.178472,0.335492,3.670619,-37.772850,w,268.77,242.053393,242.893856,86655749.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9261854,2023-02-28,2025-03-21,2023-02-08,P,450.0,57.50,62.50,0.0,75.0,0.161451,-0.714413,0.007297,154.462000,1.957872,NaN,396.26,391.240559,390.285185,96141367.0,752
9261855,2023-02-28,2025-03-21,2023-01-09,P,455.0,61.00,66.00,0.0,0.0,0.158546,-0.758524,0.007626,151.188600,2.874898,NaN,396.26,391.240559,390.285185,96141367.0,752
9261856,2023-02-28,2025-03-21,2023-01-09,P,460.0,64.50,69.50,0.0,14.0,0.154637,-0.805469,0.007817,123.774400,4.337152,NaN,396.26,391.240559,390.285185,96141367.0,752
9261857,2023-02-28,2025-03-21,2023-01-09,P,465.0,68.50,73.50,0.0,21.0,0.152118,-0.846083,0.007509,79.376680,6.359011,NaN,396.26,391.240559,390.285185,96141367.0,752


In [17]:
filtered_df1 = options_df1[(options_df1['TTE'] >= 27) & (options_df1['TTE'] <= 33)]
filtered_df1

,date,exdate,last_date,cp_flag,strike_price,best_bid,best_offer,volume,open_interest,impl_volatility,delta,gamma,vega,theta,expiry_indicator,close,adj_open,adj_close,adj_volume,TTE
828,2018-01-02,2018-01-31,2017-12-28,C,240.0,29.06,29.26,0.0,12.0,0.193414,0.983311,0.002835,3.151210,-7.538074,w,268.77,242.053393,242.893856,86655749.0,29
829,2018-01-02,2018-01-31,2017-12-28,C,245.0,24.12,24.32,0.0,2.0,0.175939,0.972425,0.004764,4.816286,-9.064763,w,268.77,242.053393,242.893856,86655749.0,29
830,2018-01-02,2018-01-31,2017-12-28,C,250.0,19.20,19.38,0.0,152.0,0.153960,0.957185,0.007811,6.907079,-10.439730,w,268.77,242.053393,242.893856,86655749.0,29
831,2018-01-02,2018-01-31,2017-12-28,C,255.0,14.33,14.52,0.0,52.0,0.133378,0.926605,0.013784,10.547560,-12.556030,w,268.77,242.053393,242.893856,86655749.0,29
832,2018-01-02,2018-01-31,2017-12-28,C,260.0,9.58,9.74,0.0,31.0,0.110864,0.868021,0.025454,16.200190,-14.825650,w,268.77,242.053393,242.893856,86655749.0,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9258523,2023-02-28,2023-03-31,2023-02-28,P,411.0,16.09,17.26,102.0,906.0,0.127514,-0.854706,0.016339,25.346960,-6.080472,m,396.26,391.240559,390.285185,96141367.0,31
9258524,2023-02-28,2023-03-31,2023-02-28,P,412.0,16.82,18.00,5.0,2911.0,0.121468,-0.882473,0.014863,21.527260,-1.950742,m,396.26,391.240559,390.285185,96141367.0,31
9258525,2023-02-28,2023-03-31,2023-02-28,P,413.0,17.57,18.77,7.0,2126.0,0.114016,-0.911503,0.012890,17.372680,2.619810,m,396.26,391.240559,390.285185,96141367.0,31
9258526,2023-02-28,2023-03-31,2023-02-28,P,414.0,18.34,19.56,1.0,1120.0,0.103528,-0.943419,0.009995,11.793440,7.923930,m,396.26,391.240559,390.285185,96141367.0,31


## More detailed stuff

Need strikes for each day to cover range of previous 30 days of stock movements; Look at C/P separately; Need ATM option to have data for next 21-30 days

In [18]:
def find_closest_tte(group, days = 30):
    # Calculate the absolute difference from days, subtract a small fraction for values less than days to prioritize lower values in case of a tie
    group['diff'] = (group['TTE'] - days).abs() - group['TTE'].lt(days) * 0.1
    # Find the row with the minimum difference
    return group.loc[group['diff'].idxmin()]

In [19]:
puts_df = filtered_df[filtered_df['cp_flag'] == 'P']
calls_df = filtered_df[filtered_df['cp_flag'] == 'C']

puts_df1 = filtered_df1[filtered_df1['cp_flag'] == 'P']
calls_df1 = filtered_df1[filtered_df1['cp_flag'] == 'C']

In [21]:
puts_result_df = puts_df.groupby('date', as_index=False).apply(find_closest_tte).reset_index(drop=True)[['date', 'TTE']].drop_duplicates()
calls_result_df = calls_df.groupby('date', as_index=False).apply(find_closest_tte).reset_index(drop=True)[['date', 'TTE']].drop_duplicates()

puts_result_df1 = puts_df1.groupby('date', as_index=False).apply(find_closest_tte).reset_index(drop=True)[['date', 'TTE']].drop_duplicates()
calls_result_df1 = calls_df1.groupby('date', as_index=False).apply(find_closest_tte).reset_index(drop=True)[['date', 'TTE']].drop_duplicates()

In [39]:
calls_result_df

,date,TTE
0,2018-01-02,29
1,2018-01-03,30
2,2018-01-04,29
3,2018-01-05,28
4,2018-01-08,30
...,...,...
1293,2023-02-22,30
1294,2023-02-23,29
1295,2023-02-24,28
1296,2023-02-27,32


In [22]:
calls_result_df1.loc[calls_result_df1['TTE'] < 28]

,date,TTE


In [23]:
set(options_df['date']) - set(calls_result_df['date'])

set()

In [24]:
# Step 1: Store the DataFrames in dictionaries
original_dfs = {'puts': puts_df, 'calls': calls_df, 'puts1': puts_df1, 'calls1': calls_df1}
result_dfs = {'puts': puts_result_df, 'calls': calls_result_df, 'puts1': puts_result_df1, 'calls1': calls_result_df1}

# Step 2: Create a new dictionary for the filtered DataFrames
filtered_dfs = {}

# Step 3: Iterate and filter based on 'date' and 'TTE'
for key in original_dfs.keys():
    # Get the DataFrame of interest and the corresponding result DataFrame
    df = original_dfs[key]
    result_df = result_dfs[key]
    
    # Filter the original DataFrame to include rows with matching 'date' and 'TTE_days' from result_df
    filtered_df = df.merge(result_df, on=['date', 'TTE'])
    
    # Store the filtered DataFrame in the new dictionary
    filtered_dfs[key] = filtered_df

In [25]:
filtered_dfs['puts']

,date,exdate,last_date,cp_flag,strike_price,best_bid,best_offer,volume,open_interest,impl_volatility,delta,gamma,vega,theta,expiry_indicator,close,adj_open,adj_close,adj_volume,TTE
0,2018-01-02,2018-01-31,NaN,P,165.0,0.00,0.01,0.0,0.0,0.535503,-0.000456,0.000040,0.123239,-0.414822,w,268.77,242.053393,242.893856,86655749.0,29
1,2018-01-02,2018-01-31,2018-01-02,P,240.0,0.13,0.14,147.0,1282.0,0.205954,-0.022718,0.003462,4.080427,-5.205224,w,268.77,242.053393,242.893856,86655749.0,29
2,2018-01-02,2018-01-31,2018-01-02,P,245.0,0.17,0.18,66.0,182.0,0.182066,-0.031889,0.005197,5.423385,-6.083812,w,268.77,242.053393,242.893856,86655749.0,29
3,2018-01-02,2018-01-31,2018-01-02,P,250.0,0.24,0.25,269.0,166.0,0.159122,-0.048143,0.008314,7.569383,-7.395249,w,268.77,242.053393,242.893856,86655749.0,29
4,2018-01-02,2018-01-31,2018-01-02,P,255.0,0.36,0.37,59.0,749.0,0.135986,-0.077530,0.014119,10.996340,-9.096192,w,268.77,242.053393,242.893856,86655749.0,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128920,2023-02-28,2023-03-31,2022-06-23,P,570.0,172.26,173.45,0.0,0.0,NaN,NaN,NaN,NaN,NaN,m,396.26,391.240559,390.285185,96141367.0,31
128921,2023-02-28,2023-03-31,2022-11-18,P,575.0,177.25,178.44,0.0,0.0,NaN,NaN,NaN,NaN,NaN,m,396.26,391.240559,390.285185,96141367.0,31
128922,2023-02-28,2023-03-31,2022-06-23,P,580.0,182.23,183.43,0.0,0.0,NaN,NaN,NaN,NaN,NaN,m,396.26,391.240559,390.285185,96141367.0,31
128923,2023-02-28,2023-03-31,2022-11-15,P,585.0,187.22,188.42,0.0,0.0,NaN,NaN,NaN,NaN,NaN,m,396.26,391.240559,390.285185,96141367.0,31


In [27]:
def select_row_with_smallest_diff(group):
    # Calculate the absolute difference
    group['abs_diff'] = (group['close'] - group['strike_price']).abs()
    # Return the row with the smallest difference
    return group.loc[group['abs_diff'].idxmin()]

# Iterate through the filtered_dfs dictionary, applying the function to each DataFrame
final_dfs = {}
for key, df in filtered_dfs.items():
    # Group by 'date' and apply the custom function with include_groups=False
    final_df = df.groupby('date', as_index=False).apply(select_row_with_smallest_diff).reset_index(drop=True)
    # Drop the 'abs_diff' column as it's no longer needed
    final_df = final_df.drop(columns=['abs_diff'])
    # Store the result in the final_dfs dictionary
    final_dfs[key] = final_df

In [28]:
final_dfs['calls1']

,date,exdate,last_date,cp_flag,strike_price,best_bid,best_offer,volume,open_interest,impl_volatility,delta,gamma,vega,theta,expiry_indicator,close,adj_open,adj_close,adj_volume,TTE
0,2018-01-02,2018-01-31,2018-01-02,C,269.0,2.11,2.16,477.0,37.0,0.068871,0.512031,0.076400,30.21236,-15.22501,w,268.77,242.053393,242.893856,86655749.0,29
1,2018-01-03,2018-02-02,2018-01-03,C,270.0,2.65,2.69,289.0,719.0,0.072398,0.562218,0.070177,30.55966,-15.81073,w,270.47,243.065564,244.430187,90070416.0,30
2,2018-01-04,2018-02-02,2018-01-04,C,272.0,2.13,2.17,769.0,6178.0,0.071179,0.500445,0.073186,30.54542,-15.79170,w,271.61,245.089905,245.460432,80595402.0,29
3,2018-01-05,2018-02-02,2018-01-05,C,273.0,2.60,2.66,1387.0,1307.0,0.074068,0.557338,0.070366,29.90136,-16.79527,w,273.42,246.273783,247.096172,83468662.0,28
4,2018-01-08,2018-02-07,2018-01-08,C,275.0,1.93,1.94,924.0,3846.0,0.072545,0.454108,0.069544,31.12119,-15.69601,w,273.92,246.996762,247.548034,57288979.0,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1293,2023-02-22,2023-03-24,2023-02-22,C,399.0,9.08,9.12,1875.0,344.0,0.202075,0.517410,0.018019,45.10776,-66.59158,w,398.54,393.496030,392.530807,83574386.0,30
1294,2023-02-23,2023-03-24,2023-02-23,C,401.0,8.79,8.82,936.0,923.0,0.196720,0.518997,0.018739,44.17986,-66.44084,w,400.66,395.505271,394.618841,95842681.0,29
1295,2023-02-24,2023-03-24,2023-02-24,C,396.0,9.05,9.08,1151.0,225.0,0.201088,0.531177,0.018872,43.16854,-68.26598,w,396.38,389.457850,390.403375,108144866.0,28
1296,2023-02-27,2023-03-31,2023-02-27,C,398.0,8.92,8.96,976.0,5244.0,0.194675,0.511410,0.018207,46.44293,-62.65579,m,397.73,393.840753,391.733020,80318244.0,32


In [29]:
for key, df in final_dfs.items():
    # Ensure 'date' is in datetime format
    df['date'] = pd.to_datetime(df['date'])

    # Extract the 'date' column as a NumPy array
    dates_np = df['date'].values

    # Create a target dates array, 21 days ahead of each date
    target_dates = dates_np + np.timedelta64(21, 'D')

    # Calculate the absolute differences between each date and every other date, using broadcasting
    abs_diff_matrix = np.abs(dates_np[:, None] - target_dates)

    # Find the index of the minimum difference for each target date
    min_diff_indices = np.argmin(abs_diff_matrix, axis=0)

    # Extract the closest dates based on the indices
    closest_dates = dates_np[min_diff_indices]

    # Assign the closest dates back to the DataFrame
    df['close_date'] = closest_dates

    # Update the DataFrame in final_dfs
    final_dfs[key] = df

In [30]:
final_dfs['calls1']

,date,exdate,last_date,cp_flag,strike_price,best_bid,best_offer,volume,open_interest,impl_volatility,...,gamma,vega,theta,expiry_indicator,close,adj_open,adj_close,adj_volume,TTE,close_date
0,2018-01-02,2018-01-31,2018-01-02,C,269.0,2.11,2.16,477.0,37.0,0.068871,...,0.076400,30.21236,-15.22501,w,268.77,242.053393,242.893856,86655749.0,29,2018-01-23
1,2018-01-03,2018-02-02,2018-01-03,C,270.0,2.65,2.69,289.0,719.0,0.072398,...,0.070177,30.55966,-15.81073,w,270.47,243.065564,244.430187,90070416.0,30,2018-01-24
2,2018-01-04,2018-02-02,2018-01-04,C,272.0,2.13,2.17,769.0,6178.0,0.071179,...,0.073186,30.54542,-15.79170,w,271.61,245.089905,245.460432,80595402.0,29,2018-01-25
3,2018-01-05,2018-02-02,2018-01-05,C,273.0,2.60,2.66,1387.0,1307.0,0.074068,...,0.070366,29.90136,-16.79527,w,273.42,246.273783,247.096172,83468662.0,28,2018-01-26
4,2018-01-08,2018-02-07,2018-01-08,C,275.0,1.93,1.94,924.0,3846.0,0.072545,...,0.069544,31.12119,-15.69601,w,273.92,246.996762,247.548034,57288979.0,30,2018-01-29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1293,2023-02-22,2023-03-24,2023-02-22,C,399.0,9.08,9.12,1875.0,344.0,0.202075,...,0.018019,45.10776,-66.59158,w,398.54,393.496030,392.530807,83574386.0,30,2023-02-28
1294,2023-02-23,2023-03-24,2023-02-23,C,401.0,8.79,8.82,936.0,923.0,0.196720,...,0.018739,44.17986,-66.44084,w,400.66,395.505271,394.618841,95842681.0,29,2023-02-28
1295,2023-02-24,2023-03-24,2023-02-24,C,396.0,9.05,9.08,1151.0,225.0,0.201088,...,0.018872,43.16854,-68.26598,w,396.38,389.457850,390.403375,108144866.0,28,2023-02-28
1296,2023-02-27,2023-03-31,2023-02-27,C,398.0,8.92,8.96,976.0,5244.0,0.194675,...,0.018207,46.44293,-62.65579,m,397.73,393.840753,391.733020,80318244.0,32,2023-02-28


In [31]:
# Convert 'date' in options_df1 and 'close_date' in each DataFrame in final_dfs to datetime if not already
options_df1['date'] = pd.to_datetime(options_df1['date'])

# Ensure 'ex_date' or 'exdate', 'cp_flag', and 'strike_price' columns are of appropriate types in options_df1
options_df1['exdate'] = pd.to_datetime(options_df1['exdate'])  # Adjusting based on actual column name

for key, df in final_dfs.items():
    df['close_date'] = pd.to_datetime(df['close_date'])
    df['exdate'] = pd.to_datetime(df['exdate'])  # Adjusting based on actual column name in final_dfs if different
    
    # Perform the merge with options_df1
    merged_df = pd.merge(df, options_df1, left_on=['close_date', 'cp_flag', 'strike_price', 'exdate'], 
                         right_on=['date', 'cp_flag', 'strike_price', 'exdate'], 
                         how='left', indicator=True)
    
    # Create a new column 'is_present' based on the merge indicator
    merged_df['is_present'] = merged_df['_merge'] == 'both'
    
    # Drop columns from options_df1 and the merge indicator, retaining only original columns from df
    columns_to_drop = ['_merge'] + [col for col in merged_df.columns if col.endswith('_y')]
    merged_df.drop(columns=columns_to_drop, inplace=True, errors='ignore')
    
    # Update the DataFrame in final_dfs
    final_dfs[key] = merged_df

In [32]:
len(final_dfs['calls1'].loc[final_dfs['calls1']['is_present'] == False])

48

In [33]:
len(final_dfs['calls'].loc[final_dfs['calls']['is_present'] == False])

48

In [34]:
len(final_dfs['puts'].loc[final_dfs['puts']['is_present'] == False])

110

In [35]:
len(final_dfs['puts1'].loc[final_dfs['puts1']['is_present'] == False])

110

In [45]:
final_dfs['puts1']

,date_x,exdate,last_date_x,cp_flag,strike_price,best_bid_x,best_offer_x,volume_x,open_interest_x,impl_volatility_x,...,vega_x,theta_x,expiry_indicator_x,close_x,adj_open_x,adj_close_x,adj_volume_x,TTE_x,close_date,is_present
0,2018-01-02,2018-01-31,2018-01-02,P,269.0,2.05,2.10,198.0,33.0,0.069577,...,30.11151,-11.46749,w,268.77,242.053393,242.893856,86655749.0,29,2018-01-23,True
1,2018-01-03,2018-02-02,2018-01-03,P,270.0,1.77,1.80,1060.0,330.0,0.069568,...,30.52431,-11.33461,w,270.47,243.065564,244.430187,90070416.0,30,2018-01-24,True
2,2018-01-04,2018-02-02,2018-01-04,P,272.0,2.12,2.15,416.0,304.0,0.068201,...,30.40449,-11.28616,w,271.61,245.089905,245.460432,80595402.0,29,2018-01-25,True
3,2018-01-05,2018-02-02,2018-01-05,P,273.0,1.81,1.85,509.0,149.0,0.071740,...,29.86764,-12.34094,w,273.42,246.273783,247.096172,83468662.0,28,2018-01-26,True
4,2018-01-08,2018-02-07,2018-01-08,P,275.0,2.53,2.56,82.0,269.0,0.067924,...,30.79770,-10.80407,w,273.92,246.996762,247.548034,57288979.0,30,2018-01-29,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1293,2023-02-22,2023-03-24,2023-02-22,P,399.0,9.67,9.71,2678.0,1317.0,0.201626,...,45.30915,-47.38424,w,398.54,393.496030,392.530807,83574386.0,30,2023-02-28,True
1294,2023-02-23,2023-03-24,2023-02-23,P,401.0,8.83,8.86,1233.0,1566.0,0.186139,...,44.77837,-44.24160,w,400.66,395.505271,394.618841,95842681.0,29,2023-02-28,True
1295,2023-02-24,2023-03-24,2023-02-24,P,396.0,8.81,8.84,3156.0,1086.0,0.198990,...,43.53449,-48.60954,w,396.38,389.457850,390.403375,108144866.0,28,2023-02-28,True
1296,2023-02-27,2023-03-31,2023-02-27,P,398.0,9.23,9.26,1568.0,4088.0,0.188579,...,46.60089,-42.60824,m,397.73,393.840753,391.733020,80318244.0,32,2023-02-28,True


In [38]:
for key, df in final_dfs.items():
    csv_filename = f"{key}.csv"
    df.to_csv(csv_filename, index=False)
    print(f"DataFrame '{key}' has been saved to '{csv_filename}'")

DataFrame 'puts' has been saved to 'puts.csv'
DataFrame 'calls' has been saved to 'calls.csv'
DataFrame 'puts1' has been saved to 'puts1.csv'
DataFrame 'calls1' has been saved to 'calls1.csv'


In [47]:
def check_dates_in_calls_and_puts(calls_key, puts_key, final_dfs):
    calls_df = final_dfs[calls_key]
    puts_df = final_dfs[puts_key]

    filtered_calls = calls_df[calls_df['is_present'] == False]
    filtered_puts = puts_df[puts_df['is_present'] == False]

    calls_dates = set(filtered_calls['date_x'].unique())
    puts_dates = set(filtered_puts['date_x'].unique())

    return calls_dates.issubset(puts_dates)

calls_in_puts = check_dates_in_calls_and_puts('calls', 'puts', final_dfs)
calls1_in_puts1 = check_dates_in_calls_and_puts('calls1', 'puts1', final_dfs)

print(f"All dates in 'calls' with is_present=False are in 'puts' with is_present=False: {calls_in_puts}")
print(f"All dates in 'calls1' with is_present=False are in 'puts1' with is_present=False: {calls1_in_puts1}")

All dates in 'calls' with is_present=False are in 'puts' with is_present=False: False
All dates in 'calls1' with is_present=False are in 'puts1' with is_present=False: False


In [51]:
def count_dates_comparison(calls_key, puts_key, final_dfs):
    calls_df = final_dfs[calls_key][final_dfs[calls_key]['is_present'] == False]
    puts_df = final_dfs[puts_key][final_dfs[puts_key]['is_present'] == False]

    calls_dates = set(calls_df['date_x'].unique())
    puts_dates = set(puts_df['date_x'].unique())

    unique_in_calls = calls_dates.difference(puts_dates)
    unique_in_puts = puts_dates.difference(calls_dates)

    overlapping_dates = calls_dates.intersection(puts_dates)

    count_unique_in_calls = len(unique_in_calls)
    count_unique_in_puts = len(unique_in_puts)
    count_overlapping = len(overlapping_dates)

    return count_unique_in_calls, count_unique_in_puts, count_overlapping

count_unique_in_calls, count_unique_in_puts, count_overlapping = count_dates_comparison('calls', 'puts', final_dfs)
count_unique_in_calls1, count_unique_in_puts1, count_overlapping1 = count_dates_comparison('calls1', 'puts1', final_dfs)

print(f"Unique dates in 'calls' not in 'puts' where is_present=False: {count_unique_in_calls}")
print(f"Unique dates in 'puts' not in 'calls' where is_present=False: {count_unique_in_puts}")
print(f"Overlapping dates in 'calls' and 'puts' where is_present=False: {count_overlapping}")

print(f"Unique dates in 'calls1' not in 'puts1' where is_present=False: {count_unique_in_calls1}")
print(f"Unique dates in 'puts1' not in 'calls1' where is_present=False: {count_unique_in_puts1}")
print(f"Overlapping dates in 'calls1' and 'puts1' where is_present=False: {count_overlapping1}")

Unique dates in 'calls' not in 'puts' where is_present=False: 48
Unique dates in 'puts' not in 'calls' where is_present=False: 110
Overlapping dates in 'calls' and 'puts' where is_present=False: 0
Unique dates in 'calls1' not in 'puts1' where is_present=False: 48
Unique dates in 'puts1' not in 'calls1' where is_present=False: 110
Overlapping dates in 'calls1' and 'puts1' where is_present=False: 0


With this, unfortunately we do see that these sets are actually exclusive, so we will ignore them for now and for the final version, we'll try to fill them in with Black-Scholes calculations if they are close enough to the other provided data values and ignore these dates as starting points otherwise.